In [2]:
%load_ext autoreload
%autoreload 2

import os
import exercise3_config as config
from readers import ExtractImagePaths, ExtractTranscriptionsAsList, ExtractKeywords


transcriptions_as_list = ExtractTranscriptionsAsList()
keywords_to_search = ExtractKeywords()

# Extract paths for train and test files
train_images_numbers, train_jpg_paths, train_svg_paths = \
  ExtractImagePaths(os.path.join(config.DATA_ROOT_DIR, 'task/train.txt'))
test_images_numbers, test_jpg_paths, test_svg_paths = \
  ExtractImagePaths(os.path.join(config.DATA_ROOT_DIR, 'task/valid.txt'))

In [3]:
import cv2 as cv
import numpy as np
from PIL import Image
from image_preprocessing import CropAllWordImages


# TODO: remove this, apply OTSU. This method is here now just for the sake of testing the whole pipeline.
# Applies KMeans clustering with the given k on the image at the given path 
# and returns the output as a a PIL.Image.
def ApplyKMeansClusteringToImageFile(jpg_image_filename, k=2):
  original_image = cv.imread(jpg_image_filename)
  pixels = original_image.reshape((-1,3))
  pixels = np.float32(pixels)

  # Define criteria, number of clusters and apply KMeans.
  criteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 10, 1.0)
  _, label, center = cv.kmeans(pixels, k, None, criteria, 10, cv.KMEANS_RANDOM_CENTERS)

  # Convert back into uint8.
  center = np.uint8(center)
  final_image_pixels = center[label.flatten()]
  final_image_pixels = final_image_pixels.reshape((original_image.shape))

  return Image.fromarray(final_image_pixels)


# Applies binarization on the original image, then crops the words using the given mask.
def ExtractWordImagesFromOriginalImage(original_image_path, mask_path):
  image_after_binarization = ApplyKMeansClusteringToImageFile(original_image_path)
  return CropAllWordImages(image_after_binarization, mask_path)


# Extracts all words needed for train and test
# Output:
#  - train_words_per_image: dict containing all train words
#  - testt_words_per_image: dict containing all test words
# Entries in the output dictionaries are like dict[page_number] = list_of_words
def ExtractTrainAndTestWords(train_images_numbers, train_jpg_paths, train_svg_paths,
                             test_images_numbers, test_jpg_paths, test_svg_paths):
  train_words_per_image = {}
  # for index in range(len(train_images_numbers)):
  # TODO: iterate over all images. we can leave it like this for now for testing
  for index in range(1):
    train_words_per_image[train_images_numbers[index]] = ExtractWordImagesFromOriginalImage(train_jpg_paths[index], train_svg_paths[index])

  test_words_per_image = {}
  # for index in range(len(test_images_numbers)):
  # TODO: iterate over all images. we can leave it like this for now for testing
  for index in range(1):
    test_words_per_image[test_images_numbers[index]] = ExtractWordImagesFromOriginalImage(test_jpg_paths[index], test_svg_paths[index])

  return train_words_per_image, test_words_per_image

train_words_per_image, test_words_per_image = \
  ExtractTrainAndTestWords(train_images_numbers, train_jpg_paths, train_svg_paths,
                           test_images_numbers, test_jpg_paths, test_svg_paths)


In [4]:
from feature_extraction import Method
from word_builder import BuildTrainAndTestWords


feature_extraction_methods = [Method.BLACK_PIXEL_RATIO]
train_words, test_words = BuildTrainAndTestWords(train_words_per_image, test_words_per_image, transcriptions_as_list, feature_extraction_methods)

In [5]:
# TODO: For testing only. Remove the entire cell later

# all_train_words = train_words
# all_test_words = test_words

train_words = train_words[:7]
test_words = test_words[:7]

In [6]:
import numpy as np
from dtaidistance import dtw
from compute_distances import ComputeDistances


distances = ComputeDistances(train_words, test_words)

In [37]:
from keyword_spotting import FindClosestKnownWord


# For each test_word, find the train_word that is closest to it.
for test_word in test_words:
  test_word.closest_train_word, test_word.distance_to_closest_train_word = \
    FindClosestKnownWord(test_word, train_words, distances)

In [ ]:
from keyword_spotting import SpotKeyword


# Search keywords.
for keyword_to_search in keywords_to_search:
  spotted_test_keywords = SpotKeyword(keyword_to_search, test_words)
  if len(spotted_test_keywords):
    print(keyword_to_search)
    print(spotted_test_keywords)
    for keyword in spotted_test_keywords:
      print(keyword.distance_to_closest_train_word)
      keyword.image.show()
    input()

In [ ]:
# TODO: write output file kws.csv (slide 24)